*Last update: 22-07-06 (YY-MM-DD)*


Bloated training notebook, for easier use. Now maintained by AUX directly.

# General setup:


---



In [ ]:
#@title Check the GPU model here.

#@markdown NNSVS can train on any GPU that Colab assigns, however the K80 may be fairly slow. <br/> Colab has strict GPU limits, especially with a free account. Free accounts will often only be able to obtain a K80 at peak times.
!nvidia-smi -L


In [ ]:
#@title (Optional) Mount Your Google Drive.
#@markdown Optional, for storing your dataset directly on your drive. Keep the I/O limits in mind. <br> You can re-run this cell to remount Drive if necessary.
from google.colab import drive
print('\033[97;100m' + 'Unmounting Google Drive first, if necessary.' + '\033[0m')
drive.flush_and_unmount()
!rm -rf /content/drive
print('\033[97;100m' + 'Mounting Google Drive.' + '\033[0m')
drive.mount("/content/drive")

# Preparation:

---



In [ ]:
#@title Step 1: Install the training kit
Processor = "GPU" #param ["GPU", "CPU"]
#@markdown You need to run this cell first to install the training kit and required packages. You only need to run this cell ONCE.

from IPython.display import clear_output

%cd -q /content
%rm -rf sample_data enunu_training_kit-main nnsvs bandmat etk.zip

print('\033[97;100m' + 'Installing Prerequisites...' + '\033[0m')
!DEBIAN_FRONTEND=noninteractive apt-get install -qq p7zip-full p7zip-rar pv &> /dev/null
%pip install --upgrade pip &> /dev/null
%pip install wheel numpy cython==0.29.27 optuna "hydra-core >= 1.1.0, < 1.2.0" "hydra_colorlog >= 1.1.0" hydra-optuna-sweeper mlflow utaupy tqdm pydub pyyaml natsort &> /dev/null

print('\033[97;100m' + 'Downloading ENUNU Training Kit', Kit_Version + '\033[0m')
!wget -qO etk.zip https://github.com/oatsu-gh/enunu_training_kit/archive/refs/tags/v1.0.0.zip

!7za -bso0 x etk.zip
!mv enunu_training_kit-* enunu_training_kit-main
#patch training kit for nnsvs commit 1c5dd93ad7556f75d0735c3c63acbe31b763c5b0
if Kit_Version == "v1.0.0":
  !sed -i 's|scaler.class=$scaler_class|scaler._target_=$scaler_class|g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/feature_generation.sh
  !sed -i 's+db_root:                singing_database+db_root:                ../singing_database+g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i 's+db_root:              singing_database+db_root:              ../singing_database+g' /content/enunu_training_kit-main/train/config_rmdn.yaml
  !sed -i 's+$PYTHON_ROOT/python+python+g' /content/enunu_training_kit-main/train/run.sh
  !sed -i 's+$PYTHON_ROOT/python+python+g' /content/enunu_training_kit-main/train/run_rmdn.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/feature_generation.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/train_timelag.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/train_duration.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/train_acoustic.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/generate.sh
  !sed -i 's+$PYTHON_EXE+python+g' /content/enunu_training_kit-main/train/nnsvs_shell_scripts/synthesis.sh

!sed -i 's+NNSVS_ROOT=../nnsvs+NNSVS_ROOT=/content/nnsvs+g' /content/enunu_training_kit-main/train/run.sh
!sed -i 's+NNSVS_ROOT="../nnsvs"+NNSVS_ROOT="/content/nnsvs"+g' /content/enunu_training_kit-main/train/run.sh

train_acoustic = ["# @package _group_","log_dir: tensorboard/exp/acoustic","","out_dir:                    exp","max_train_steps:            -1","nepochs:                    50","checkpoint_epoch_interval:  20","","feats_criterion: mse","","stream_wise_loss:           false","use_detect_anomaly:         false","","use_amp: true","","pitch_reg_weight: 1.0","force_fix_vuv: true","","optim:","  optimizer:","    name:                   RAdam","    params:","      lr:                   0.001","      betas:                [0.9, 0.999]","      weight_decay:         0.0","  lr_scheduler:","    name:                   StepLR","    params:","      step_size:            20","      gamma:                0.5","","resume:","  checkpoint:","  load_optimizer:           false","","cudnn:","  benchmark:                false","  deterministic:            true",""]
with open("/content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml", "w") as config:
  config.writelines('\n'.join(train_acoustic))
train_duration = ["# @package _group_","log_dir: tensorboard/exp/duration","","out_dir:                    exp","max_train_steps:            -1","nepochs:                    50","checkpoint_epoch_interval:  20","","feats_criterion: mse","","stream_wise_loss:           false","use_detect_anomaly:         false","","use_amp: true","","optim:","  optimizer:","    name:                   RAdam","    params:","      lr:                   0.001","      betas:                [0.9, 0.999]","      weight_decay:         0.0","  lr_scheduler:","    name:                   StepLR","    params:","      step_size:            20","      gamma:                0.5","","resume:","  checkpoint:","  load_optimizer:           false","","cudnn:","  benchmark:                false","  deterministic:            true",""]
with open("/content/enunu_training_kit-main/train/conf/train/duration/train/myconfig.yaml", "w") as config:
  config.writelines('\n'.join(train_duration))
train_timelag = ["# @package _group_","log_dir: tensorboard/exp/timelag","","out_dir:                    exp","max_train_steps:            -1","nepochs:                    50","checkpoint_epoch_interval:  20","","feats_criterion: mse","","stream_wise_loss:           false","use_detect_anomaly:         false","","use_amp: true","","optim:","  optimizer:","    name:                   RAdam","    params:","      lr:                   0.001","      betas:                [0.9, 0.999]","      weight_decay:         0.0","  lr_scheduler:","    name:                   StepLR","    params:","      step_size:            20","      gamma:                0.5","","resume:","  checkpoint:","  load_optimizer:           false","","cudnn:","  benchmark:                false","  deterministic:            true",""]
with open("/content/enunu_training_kit-main/train/conf/train/timelag/train/myconfig.yaml", "w") as config:
  config.writelines('\n'.join(train_timelag))
vib_enuconfig = ["# Copyright (c) 2020 Ryuichi Yamamoto","# Copyright (c) 2020 oatsu","","# Is trained with ENUNU specific recipe","trained_for_enunu:      true","# Table (lyric -> phonemes) file path.","table_path:             dic/kana2phonemes_002_oto2lab.table","","defaults:","    - hydra/job_logging: colorlog","    - hydra/hydra_logging: colorlog","","verbose:                100","","# Setting for output WAV file.","sample_rate:            44100","gain_normalize:         false","","# How was the model trained.","frame_period:           5","question_path:          hed/jp_qst_crazy_mono_013_enunu_221D.hed","log_f0_conditioning:    true","vibrato_scale: 1.0","","# Use ground truth duration or not","# if true, time-lag and duration models will not be used.","ground_truth_duration:  false","","","# If not empty, try to search statisics in the directory","stats_dir:              dump/Unnamed/norm","# If not empty, try to search models in the directory","model_dir:              exp/Unnamed","","acoustic:","    question_path:      null","    checkpoint:         best_loss.pth","    in_scaler_path:     null","    out_scaler_path:    null","    # model_yaml:","    subphone_features:  coarse_coding","    relative_f0:        true","    post_filter:        true","","duration:","    checkpoint:         best_loss.pth","    question_path:      null","    in_scaler_path:     null","    out_scaler_path:    null","","timelag:","    question_path:      null","    checkpoint:         best_loss.pth","    in_scaler_path:     null","    out_scaler_path:    null","    allowed_range:      [-150, 150]","    allowed_range_rest: [-150, 250]",""]
with open("/content/enunu_training_kit-main/train/enuconfig.yaml", "w") as config:
  config.writelines('\n'.join(vib_enuconfig))
run_sh = ["#!/bin/bash","","##########","# customized for Portable ENUNU Training Kit on Windows","##########","# Set bash to 'debug' mode, it will exit on :","# -e 'error', -u 'undefined variable', -o ... 'error in pipeline', -x 'print commands',","set -e","set -u","set -o pipefail","","function xrun () {","    set -x","    $@","    set +x","}","","# use embed python executional file","PYTHON_ROOT=\"python-3.8.10-embed-amd64\"","PYTHON_EXE=\"python.exe\"","PYTHON_SCRIPTS_ROOT=\"$PYTHON_ROOT/Scripts\"","CONFIG_PATH=\"config.yaml\"","","script_dir=$(cd $(dirname ${BASH_SOURCE:-$0}); pwd)","","# changed-----------------------------------------------","NNSVS_ROOT=\"/content/nnsvs\"","NNSVS_COMMON_ROOT=\"/content/nnsvs/recipes/_common/spsvs\"",". \"$NNSVS_ROOT/utils/yaml_parser.sh\" || exit 1;","# to----------------------------------------------------","NNSVS_SHELL_SCRIPTS_ROOT=\"nnsvs_shell_scripts\"","#. $NNSVS_SHELL_SCRIPTS_ROOT/yaml_parser.sh || exit 1;","# ------------------------------------------------------","","eval $(parse_yaml $CONFIG_PATH \"\")","","train_set=\"train_no_dev\"","dev_set=\"dev\"","eval_set=\"eval\"","datasets=($train_set $dev_set $eval_set)","testsets=($dev_set $eval_set)","","dumpdir=dump","dump_org_dir=\"$dumpdir/$spk/org\"","dump_norm_dir=\"$dumpdir/$spk/norm\"","","stage=0","stop_stage=-1","","# changed-----------------------------------------------",". $NNSVS_ROOT/utils/parse_options.sh || exit 1;","# to----------------------------------------------------","# . $NNSVS_SHELL_SCRIPTS_ROOT/parse_options.sh || exit 1;","# ------------------------------------------------------","","# exp name","if [ -z ${tag:=} ]; then","    expname=\"${spk}\"","else","    expname=\"${spk}_${tag}\"","fi","expdir=\"exp/$expname\"","","","# Prepare files in singing-database for training","if [ ${stage} -le 0 ] && [ ${stop_stage} -ge 0 ]; then","    echo \"#########################################\"","    echo \"#                                       #\"","    echo \"#  stage 0: Data preparation            #\"","    echo \"#                                       #\"","    echo \"#########################################\"","    rm -rf $out_dir","    rm -f preprocess_data.py.log","    python preprocess_data.py $CONFIG_PATH || exit 1;","    echo \"\"","fi","","","# Analyze .wav and .lab files","if [ ${stage} -le 1 ] && [ ${stop_stage} -ge 1 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 1: Feature generation           #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    rm -rf $dumpdir","    # changed-----------------------------------------------","    # . $NNSVS_COMMON_ROOT/feature_generation.sh || exit 1;","    # to----------------------------------------------------","    . $NNSVS_COMMON_ROOT/feature_generation.sh || exit 1;","    # ------------------------------------------------------","    echo \"\"","fi","","","# Train time-lag model","if [ ${stage} -le 2 ] && [ ${stop_stage} -ge 2 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 2: Time-lag model training      #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    # changed-----------------------------------------------","    . $NNSVS_COMMON_ROOT/train_timelag.sh || exit 1;","    # to----------------------------------------------------","    #. $NNSVS_SHELL_SCRIPTS_ROOT/train_timelag.sh || exit 1;","    # ------------------------------------------------------","    echo \"\"","fi","","","# Train duration model","if [ ${stage} -le 3 ] && [ ${stop_stage} -ge 3 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 3: Duration model training      #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    # changed-----------------------------------------------","    . $NNSVS_COMMON_ROOT/train_duration.sh || exit 1;","    # to----------------------------------------------------","    # . $NNSVS_SHELL_SCRIPTS_ROOT/train_duration.sh || exit 1;","    # ------------------------------------------------------","    echo \"\"","fi","","","# Train acoustic model","if [ ${stage} -le 4 ] && [ ${stop_stage} -ge 4 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 4: Training acoustic model      #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    # changed-----------------------------------------------","    . $NNSVS_COMMON_ROOT/train_acoustic.sh || exit 1;","    # to----------------------------------------------------","    # . $NNSVS_SHELL_SCRIPTS_ROOT/train_acoustic.sh || exit 1;","    # ------------------------------------------------------","    echo \"\"","fi","","","# Generate models from timelag/duration/acoustic models","if [ ${stage} -le 5 ] && [ ${stop_stage} -ge 5 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 5: Feature generation           #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    # changed-----------------------------------------------","    . $NNSVS_COMMON_ROOT/generate.sh || exit 1;","    # to----------------------------------------------------","    # . $NNSVS_SHELL_SCRIPTS_ROOT/generate.sh || exit 1;","    # ------------------------------------------------------","    echo \"\"","fi","","#","# # Synthesis wav files","# if [ ${stage} -le 6 ] && [ ${stop_stage} -ge 6 ]; then","#     echo \"##########################################\"","#     echo \"#                                        #\"","#     echo \"#  stage 6: Waveform synthesis           #\"","#     echo \"#                                        #\"","#     echo \"##########################################\"","#     . $NNSVS_COMMON_ROOT/synthesis.sh","#     echo \"\"","# fi","#","","# Copy the models to release directory","if [ ${stage} -le 7 ] && [ ${stop_stage} -ge 7 ]; then","    echo \"##########################################\"","    echo \"#                                        #\"","    echo \"#  stage 7: Release preparation          #\"","    echo \"#                                        #\"","    echo \"##########################################\"","    python prepare_release.py $CONFIG_PATH || exit 1;","    echo \"\"","fi"]
with open("/content/enunu_training_kit-main/train/run.sh", "w") as config:
  config.writelines('\n'.join(run_sh))
!echo "sample_rate: 44100" >> /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml

print('\033[97;100m' + 'Installing NNSVS' + '\033[0m')
%pip install -I git+https://github.com/r9y9/nnsvs.git &> /dev/null
!git clone https://github.com/r9y9/nnsvs &> /dev/null

train_acoustic_sh = ["if [ -d conf/train ]; then","    ext=\"--config-dir conf/train/acoustic\"","else","    ext=\"\"","fi","","if [ ! -z \"${pretrained_expdir}\" ]; then","    resume_checkpoint=$pretrained_expdir/${acoustic_model}/latest.pth","else","    resume_checkpoint=","fi","","if [[ ${acoustic_hydra_optuna_sweeper_args+x} && ! -z $acoustic_hydra_optuna_sweeper_args ]]; then","    hydra_opt=\"-m ${acoustic_hydra_optuna_sweeper_args}\"","    post_args=\"mlflow.enabled=true mlflow.experiment=${expname}_${acoustic_model} hydra.sweeper.n_trials=${acoustic_hydra_optuna_sweeper_n_trials}\"","else","    hydra_opt=\"\"","    post_args=\"\"","fi","","xrun nnsvs-train $ext $hydra_opt \\","    +data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    +data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    model=$acoustic_model train=$acoustic_train data=$acoustic_data \\","    data.train_no_dev.in_dir=$dump_norm_dir/$train_set/in_acoustic/ \\","    data.train_no_dev.out_dir=$dump_norm_dir/$train_set/out_acoustic/ \\","    data.dev.in_dir=$dump_norm_dir/$dev_set/in_acoustic/ \\","    data.dev.out_dir=$dump_norm_dir/$dev_set/out_acoustic/ \\","    data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    train.out_dir=$expdir/acoustic \\","    train.resume.checkpoint=$resume_checkpoint $post_args"]
with open("/content/nnsvs/recipes/_common/spsvs/train_acoustic.sh", "w") as config:
  config.writelines('\n'.join(train_acoustic_sh))
train_timelag_sh = ["if [ -d conf/train ]; then","    ext=\"--config-dir conf/train/timelag\"","else","    ext=\"\"","fi","","if [ ! -z \"${pretrained_expdir}\" ]; then","    resume_checkpoint=$pretrained_expdir/${timelag_model}/latest.pth","else","    resume_checkpoint=","fi","","if [[ ${timelag_hydra_optuna_sweeper_args+x} && ! -z $timelag_hydra_optuna_sweeper_args ]]; then","    hydra_opt=\"-m ${timelag_hydra_optuna_sweeper_args}\"","    post_args=\"mlflow.enabled=true mlflow.experiment=${expname}_${timelag_model} hydra.sweeper.n_trials=${timelag_hydra_optuna_sweeper_n_trials}\"","else","    hydra_opt=\"\"","    post_args=\"\"","fi","","xrun nnsvs-train $ext $hydra_opt \\","    +data.in_scaler_path=$dump_norm_dir/in_timelag_scaler.joblib \\","    +data.out_scaler_path=$dump_norm_dir/out_timelag_scaler.joblib \\","    model=$timelag_model train=$timelag_train data=$timelag_data \\","    data.train_no_dev.in_dir=$dump_norm_dir/$train_set/in_timelag/ \\","    data.train_no_dev.out_dir=$dump_norm_dir/$train_set/out_timelag/ \\","    data.dev.in_dir=$dump_norm_dir/$dev_set/in_timelag/ \\","    data.dev.out_dir=$dump_norm_dir/$dev_set/out_timelag/ \\","    data.in_scaler_path=$dump_norm_dir/in_timelag_scaler.joblib \\","    data.out_scaler_path=$dump_norm_dir/out_timelag_scaler.joblib \\","    train.out_dir=$expdir/timelag \\","    train.resume.checkpoint=$resume_checkpoint $post_args"]
with open("/content/nnsvs/recipes/_common/spsvs/train_timelag.sh", "w") as config:
  config.writelines('\n'.join(train_timelag_sh))
train_duration_sh = ["if [ -d conf/train ]; then","    ext=\"--config-dir conf/train/duration\"","else","    ext=\"\"","fi","","if [ ! -z \"${pretrained_expdir}\" ]; then","    resume_checkpoint=$pretrained_expdir/${duration_model}/latest.pth","else","    resume_checkpoint=","fi","","if [[ ${duration_hydra_optuna_sweeper_args+x} && ! -z $duration_hydra_optuna_sweeper_args ]]; then","    hydra_opt=\"-m ${duration_hydra_optuna_sweeper_args}\"","    post_args=\"mlflow.enabled=true mlflow.experiment=${expname}_${duration_model} hydra.sweeper.n_trials=${duration_hydra_optuna_sweeper_n_trials}\"","else","    hydra_opt=\"\"","    post_args=\"\"","fi","","xrun nnsvs-train $ext $hydra_opt \\","    +data.in_scaler_path=$dump_norm_dir/in_duration_scaler.joblib \\","    +data.out_scaler_path=$dump_norm_dir/out_duration_scaler.joblib \\","    model=$duration_model train=$duration_train data=$duration_data \\","    data.train_no_dev.in_dir=$dump_norm_dir/$train_set/in_duration/ \\","    data.train_no_dev.out_dir=$dump_norm_dir/$train_set/out_duration/ \\","    data.dev.in_dir=$dump_norm_dir/$dev_set/in_duration/ \\","    data.dev.out_dir=$dump_norm_dir/$dev_set/out_duration/ \\","    data.in_scaler_path=$dump_norm_dir/in_duration_scaler.joblib \\","    data.out_scaler_path=$dump_norm_dir/out_duration_scaler.joblib \\","    train.out_dir=$expdir/duration \\","    train.resume.checkpoint=$resume_checkpoint $post_args"]
with open("/content/nnsvs/recipes/_common/spsvs/train_duration.sh", "w") as config:
  config.writelines('\n'.join(train_duration_sh))
generate_sh = ["for s in ${testsets[@]}; do","    for typ in timelag duration acoustic; do","        if [ $typ = \"timelag\" ]; then","            eval_checkpoint=$timelag_eval_checkpoint","            model=timelag","        elif [ $typ = \"duration\" ]; then","            eval_checkpoint=$duration_eval_checkpoint","            model=duration","        else","            eval_checkpoint=$acoustic_eval_checkpoint","            model=acoustic","        fi","","        checkpoint=$expdir/$model/${eval_checkpoint}","        name=$(basename $checkpoint)","        xrun nnsvs-generate model.checkpoint=$checkpoint \\","            model.model_yaml=$expdir/$model/model.yaml \\","            out_scaler_path=$dump_norm_dir/out_${typ}_scaler.joblib \\","            in_dir=$dump_norm_dir/$s/in_${typ}/ \\","            out_dir=$expdir/$model/predicted/$s/${name%.*}/","    done","done",""]
with open("/content/nnsvs/recipes/_common/spsvs/generate.sh", "w") as config:
  config.writelines('\n'.join(generate_sh))

print('\033[97;100m' + 'Installing bandmat' + '\033[0m')
%pip install git+https://github.com/MattShannon/bandmat.git &> /dev/null

print('\033[97;100m' + 'Updating Pytorch' + '\033[0m')
if Processor == "GPU":
  %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 &> /dev/null
else:
  %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu &> /dev/null

%load_ext tensorboard

print('Done')

In [ ]:
#@title Step 2: Decompress your dataset:

#@markdown 7z and rar files suggested.
#@markdown Uploading to colab will fail if your dataset is large, use Google Drive.

#@markdown The compressed file should contain all the files for your dataset. <u>File name is case-sensitive.</u>
empty_dataset_folder = False #@param {type:"boolean"}
#@markdown Check this if you are training separate datasets in the same folder.
compressed_dataset_path = "/content/dataset.*" #@param {type:"string"}

if empty_dataset_folder == True:
  !rm -rf /content/enunu_training_kit-main/singing_database

!7za  -bso0 -y x "$compressed_dataset_path" -o/content/enunu_training_kit-main/singing_database/
print('\nDone!')

#@markdown If you choose to not use this cell, upload your files to `enunu_training_kit-main/singing_database/`


In [ ]:
#@title Alternative: Extract dump.
#@markdown To save time in Colab, you can do feature extraction (stage 1) on your local pc and then upload that to Google Drive to train here.
#@markdown </br>Note that both your local and the notebook's config need to match or it will not work.
compressed_dump_path = "/content/dump.*" #@param {type:"string"}
#@markdown The files will be extracted to `enunu_training_kit-main/train/dump/`. The folder inside your archive needs to match the name of the singer.

!7za  -bso0 -y x "$compressed_dump_path" -o/content/enunu_training_kit-main/train/dump/
print('\nDone!')

# Change Language and Parameters (Optional)

In [ ]:
#@markdown ##(Optional) Change Langauge and Model Type

#@markdown Run this cell if you want to change any settings from their defaults. The language will automatically apply the required configuration for the selected model type.

Language = "DYVAUX Japanese v1.0.1" #@param ["DYVAUX Japanese v1.0.1", "DYVAUX Japanese MDN v1.0.1", "Default Japanese", "Default Japanese MDN", "DYVAUX English v0.7.0", "DYVAUX English MDN v0.7.0", "DYVAUX Romance v0.1.0", "SzTP's Polish v0.2.4", "SzTP's Polish MDN v0.2.4", "Archivoice Chinese v0.2.1", "Archivoice Chinese MDN v0.2.1", "Custom"]

Acoustic_Model_Type = "Conv1dResnet" #@param ["Conv1dResnet", "Conv1dResnet_use_mdn_true", "ResF0Conv1dResnet", "ResF0Conv1dResnetMDN", "ResSkipF0FFConvLSTM"]
#@markdown `ResF0FFConvLSTM` is only recommented for final datasets. </br> The VERY large size and long training times are not ideal for small and WIP datasets and won't provide a large quality improvement. It also requires a better gpu to render.
Duration_Model_Type = "LSTMRNN" #@param ["LSTMRNN", "MDN", "MDNv2"]
Timelag_Model_Type = "FeedForwardNet" #@param ["FeedForwardNet", "MDN", "MDNv2"]
#@markdown Note: Vibrato models DO NOT work in ENUNU by default.
Vibrato_Mode = "none" #@param ["none", "diff", "sine"]
#@markdown For ResF0 models, you may set the level of pitch stabilization with `pitch_reg_weight`.
Pitch_reg_weight = 0.6 #@param {type:"number"}
correct_vuv_by_phone = True
#@markdown Sample rate of the dataset.
sample_rate = "44100" #@param ["44100", "48000"]

!sed -i -r 's|(acoustic_model:)(\s+)(.+)|\1\2acoustic_custom|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(duration_model:)(\s+)(.+)|\1\2duration_custom|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(timelag_model:)(\s+)(.+)|\1\2timelag_custom|g' /content/enunu_training_kit-main/train/config.yaml

#This is a really bad way to do this but I'm lazy and don't know Python.
#At least is works without problems.

!sed -i -r 's|(pitch_reg_weight:)(\s+)(.+)|\1\2{Pitch_reg_weight}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
!sed -i -r 's|(force_fix_vuv:)(\s+)(.+)|\1\2{correct_vuv_by_phone}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
!sed -i -r 's|(sample_rate:)(\s+)(.+)|\1\2{sample_rate}|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(sample_rate:)(\s+)(.+)|\1\2{sample_rate}|g' /content/enunu_training_kit-main/train/enuconfig.yaml
!sed -i -r 's|(sample_rate:)(\s+)(.+)|\1\2{sample_rate}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
!sed -i -r 's|(sample_rate:)(\s+)(.+)|\1\2{sample_rate}|g' /content/enunu_training_kit-main/train/enuconfig.yaml

elif Language == "DYVAUX Japanese v1.0.1":
  in_dim = 368
  in_lf0_idx = 323
  in_rest_idx = 23
elif Language == "DYVAUX Japanese MDN v1.0.1":
  in_dim = 263
  in_lf0_idx = 323
  in_rest_idx = 23
elif Language == "DYVAUX English v0.7.0":
  in_dim = 434
  in_lf0_idx = 389
  in_rest_idx = 23
elif Language == "DYVAUX English MDN v0.7.0":
  in_dim = 263
  in_lf0_idx = 221
  in_rest_idx = 23
elif Language == "DYVAUX Romance v0.1.0":
  in_dim = 328
  in_lf0_idx = 283
  in_rest_idx = 22
elif Language == "SzTP's Polish v0.2.4":
  in_dim = 317
  in_lf0_idx = 272
  in_rest_idx = 11
elif Language == "SzTP's Polish MDN v0.2.4":
  in_dim = 109
  in_lf0_idx = 164
  in_rest_idx = 11
elif Language == "Archivoice Chinese v0.2.1":
  in_dim = 368
  in_lf0_idx = 323
  in_rest_idx = 8
elif Language == "Archivoice Chinese MDN v0.2.1":
  in_dim = 268
  in_lf0_idx = 223
  in_rest_idx = 8
elif Language == "Custom":
  acoustic_in_dim = 0
  in_dim = 0
  in_lf0_idx = 0

acoustic_in_dim = int(in_dim) + 4

#set the relative_f0 setting for enunu based on the acosutic model selected.
if Acoustic_Model_Type == "ResF0Conv1dResnet" or Acoustic_Model_Type == "ResF0Conv1dResnetMDN" or Acoustic_Model_Type == "ResSkipF0FFConvLSTM":
  !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2false|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  train_acoustic = ["if [ -d conf/train ]; then","    ext=\"--config-dir conf/train/acoustic\"","else","    ext=\"\"","fi","","if [ ! -z \"${pretrained_expdir}\" ]; then","    resume_checkpoint=$pretrained_expdir/${acoustic_model}/latest.pth","else","    resume_checkpoint=","fi","","if [[ ${acoustic_hydra_optuna_sweeper_args+x} && ! -z $acoustic_hydra_optuna_sweeper_args ]]; then","    hydra_opt=\"-m ${acoustic_hydra_optuna_sweeper_args}\"","    post_args=\"mlflow.enabled=true mlflow.experiment=${expname}_${acoustic_model} hydra.sweeper.n_trials=${acoustic_hydra_optuna_sweeper_n_trials}\"","else","    hydra_opt=\"\"","    post_args=\"\"","fi","","xrun nnsvs-train-resf0 $ext $hydra_opt \\","    +data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    +data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    model=$acoustic_model train=$acoustic_train data=$acoustic_data \\","    data.train_no_dev.in_dir=$dump_norm_dir/$train_set/in_acoustic/ \\","    data.train_no_dev.out_dir=$dump_norm_dir/$train_set/out_acoustic/ \\","    data.dev.in_dir=$dump_norm_dir/$dev_set/in_acoustic/ \\","    data.dev.out_dir=$dump_norm_dir/$dev_set/out_acoustic/ \\","    data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    train.out_dir=$expdir/acoustic \\","    train.resume.checkpoint=$resume_checkpoint $post_args"]
  with open("/content/nnsvs/recipes/_common/spsvs/train_acoustic.sh", "w") as config:
    config.writelines('\n'.join(train_acoustic))
  acoustic_data_conf = ["# training set","train_no_dev:","  in_dir:","  out_dir:","","# development set","dev:","  in_dir:","  out_dir:","","# data loader","num_workers: 2","batch_size: 8","pin_memory: true","","in_lf0_idx: " + str(in_lf0_idx),"out_lf0_idx: 180","in_rest_idx: " + str(in_rest_idx),"sample_rate: " + str(sample_rate)]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_data_conf))
  !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2false|g' /usr/local/lib/python3.7/dist-packages/nnsvs/bin/conf/prepare_features/acoustic/static_deltadelta.yaml
else:
  train_acoustic = ["if [ -d conf/train ]; then","    ext=\"--config-dir conf/train/acoustic\"","else","    ext=\"\"","fi","","if [ ! -z \"${pretrained_expdir}\" ]; then","    resume_checkpoint=$pretrained_expdir/${acoustic_model}/latest.pth","else","    resume_checkpoint=","fi","","if [[ ${acoustic_hydra_optuna_sweeper_args+x} && ! -z $acoustic_hydra_optuna_sweeper_args ]]; then","    hydra_opt=\"-m ${acoustic_hydra_optuna_sweeper_args}\"","    post_args=\"mlflow.enabled=true mlflow.experiment=${expname}_${acoustic_model} hydra.sweeper.n_trials=${acoustic_hydra_optuna_sweeper_n_trials}\"","else","    hydra_opt=\"\"","    post_args=\"\"","fi","","xrun nnsvs-train $ext $hydra_opt \\","    +data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    +data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    model=$acoustic_model train=$acoustic_train data=$acoustic_data \\","    data.train_no_dev.in_dir=$dump_norm_dir/$train_set/in_acoustic/ \\","    data.train_no_dev.out_dir=$dump_norm_dir/$train_set/out_acoustic/ \\","    data.dev.in_dir=$dump_norm_dir/$dev_set/in_acoustic/ \\","    data.dev.out_dir=$dump_norm_dir/$dev_set/out_acoustic/ \\","    data.in_scaler_path=$dump_norm_dir/in_acoustic_scaler.joblib \\","    data.out_scaler_path=$dump_norm_dir/out_acoustic_scaler.joblib \\","    train.out_dir=$expdir/acoustic \\","    train.resume.checkpoint=$resume_checkpoint $post_args"]
  with open("/content/nnsvs/recipes/_common/spsvs/train_acoustic.sh", "w") as config:
    config.writelines('\n'.join(train_acoustic))
  !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2true|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  acoustic_data_conf = ["# training set","train_no_dev:","  in_dir:","  out_dir:","","# development set","dev:","  in_dir:","  out_dir:","","# data loader","num_workers: 2","batch_size: 8","pin_memory: true","sample_rate: " + str(sample_rate)]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_data_conf))
  !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2true|g' /usr/local/lib/python3.7/dist-packages/nnsvs/bin/conf/prepare_features/acoustic/static_deltadelta.yaml

if Acoustic_Model_Type == "Conv1dResnet":
  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.Conv1dResnet","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 4"]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_lines))
  !sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\22|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(batch_size:)(\s+)(.+)|\1\28|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
  !sed -i -r 's|(nepochs:)(\s+)(.+)|\1\280|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
elif Acoustic_Model_Type == "Conv1dResnet_use_mdn_true":
  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.Conv1dResnet","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 4","  use_mdn: true","  num_gaussians: 8","  dim_wise: true"]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_lines))
  !sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\26|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(batch_size:)(\s+)(.+)|\1\22|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
  !sed -i -r 's|(nepochs:)(\s+)(.+)|\1\2150|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
elif Acoustic_Model_Type == "ResF0Conv1dResnet":
#  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.ResF0Conv1dResnet","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 6","","  in_lf0_idx: " + str(in_lf0_idx),"  out_lf0_idx: 180","  in_lf0_min: " + str(in_lf0_min),"  in_lf0_max: " + str(in_lf0_max),"  out_lf0_mean: " + str(out_lf0_mean),"  out_lf0_scale: " + str(out_lf0_scale)]
  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.ResF0Conv1dResnet","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 4","","  in_lf0_idx: " + str(in_lf0_idx),"  out_lf0_idx: 180","  in_lf0_min: null","  in_lf0_max: null","  out_lf0_mean: null","  out_lf0_scale: null"]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_lines))
  !sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\22|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(batch_size:)(\s+)(.+)|\1\28|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
  !sed -i -r 's|(nepochs:)(\s+)(.+)|\1\280|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
#  print("This isn't fully set up. Read the instructions for how to finish setting it up!")
elif Acoustic_Model_Type == "ResF0Conv1dResnetMDN":
#  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.ResF0Conv1dResnetMDN","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 6","  num_gaussians: 4","  dim_wise: true","","  in_lf0_idx: " + str(in_lf0_idx),"  out_lf0_idx: 180","  in_lf0_min: " + str(in_lf0_min),"  in_lf0_max: " + str(in_lf0_max),"  out_lf0_mean: " + str(out_lf0_mean),"  out_lf0_scale: " + str(out_lf0_scale)]
  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.ResF0Conv1dResnetMDN","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  hidden_dim: 128","  num_layers: 4","  num_gaussians: 4","  dim_wise: true","","  in_lf0_idx: " + str(in_lf0_idx),"  out_lf0_idx: 180","  in_lf0_min: null","  in_lf0_max: null","  out_lf0_mean: null","  out_lf0_scale: null"]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_lines))
  !sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\26|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(batch_size:)(\s+)(.+)|\1\22|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
  !sed -i -r 's|(nepochs:)(\s+)(.+)|\1\2150|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
#  print("This isn't fully set up. Read the instructions for how to finish setting it up!")
elif Acoustic_Model_Type == "ResSkipF0FFConvLSTM":
  acoustic_lines = ["# (mgc, lf0, vuv, bap)","stream_sizes: [180, 3, 1, 15]","has_dynamic_features: [true, true, false, true]","num_windows: 3","# If None, automatically set based on stream sizes","stream_weights:","","netG:","  _target_: nnsvs.model.ResSkipF0FFConvLSTM","  in_dim: " + str(acoustic_in_dim),"  out_dim: 199","  ff_hidden_dim: 2048","  conv_hidden_dim: 1024","  lstm_hidden_dim: 256","  num_lstm_layers: 2","","  in_lf0_idx: " + str(in_lf0_idx),"  out_lf0_idx: 180","  in_lf0_min: null","  in_lf0_max: null","  out_lf0_mean: null","  out_lf0_scale: null"]
  with open("/content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml", "w") as config:
    config.writelines('\n'.join(acoustic_lines))
  !sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\22|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(batch_size:)(\s+)(.+)|\1\28|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
  !sed -i -r 's|(nepochs:)(\s+)(.+)|\1\280|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
else:
  print("yeet")


if Duration_Model_Type == "LSTMRNN": 
  duration_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.LSTMRNN","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 64","  num_layers: 2","  bidirectional: true",""]
  with open("/content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml", "w") as config: 
    config.writelines('\n'.join(duration_lines))
elif Duration_Model_Type == "MDN": 
  duration_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.MDN","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 256","  num_layers: 3","  num_gaussians: 3","  dim_wise: true"]
  with open("/content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml", "w") as config: 
    config.writelines('\n'.join(duration_lines))
elif Duration_Model_Type == "MDNv2": 
  duration_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.MDN","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 256","  num_layers: 3","  dropout: 0.5","  num_gaussians: 3","  dim_wise: true"]
  with open("/content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml", "w") as config: 
    config.writelines('\n'.join(duration_lines))

if Timelag_Model_Type == "FeedForwardNet": 
  timelag_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.FeedForwardNet","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 128","  num_layers: 2",""]
  with open("/content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml", "w") as config: 
    config.writelines('\n'.join(timelag_lines))
elif Timelag_Model_Type == "MDN": 
  timelag_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.MDN","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 64","  num_layers: 3","  num_gaussians: 3","  dim_wise: true"]
  with open("/content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml", "w") as config: 
    config.writelines('\n'.join(timelag_lines))
elif Timelag_Model_Type == "MDNv2": 
  timelag_lines = ["stream_sizes: [1]","has_dynamic_features: [false]","stream_weights: [1]","","netG: ","  _target_: nnsvs.model.MDN","  in_dim: " + str(in_dim),"  out_dim: 1","  hidden_dim: 64","  num_layers: 3","  dropout: 0.5","  num_gaussians: 3","  dim_wise: true"]
  with open("/content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml", "w") as config: 
    config.writelines('\n'.join(timelag_lines))

#bullshit to change languages as needed
%cd -q /content
if Language == "DYVAUX English v0.7.0":
  !rm -rf nnsvs-english-support-*
  !wget -O nnsvs-english-support-v0.7.0.zip https://github.com/DYVAUX/nnsvs-english-support/archive/refs/tags/v0.7.0.zip
  !unzip nnsvs-english-support-v0.7.0.zip
  !mv nnsvs-english-support-*/ nnsvs-english-support-main/
  !cp /content/nnsvs-english-support-main/hed/DVX_English_NNSVS.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-english-support-main/dic/blank.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_English_NNSVS.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'ah', 'aa', 'ao', 'ae', 'ax', 'iy', 'ih', 'uh', 'uw', 'eh', 'er', 'ay', 'a', 'ey', 'e', 'oy', 'aw', 'ow', 'o', 'el', 'em', 'en'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_English_NNSVS.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif Language == "DYVAUX English MDN v0.7.0":
  !rm -rf nnsvs-english-support-*
  !wget -O nnsvs-english-support-v0.7.0.zip https://github.com/DYVAUX/nnsvs-english-support/archive/refs/tags/v0.7.0.zip
  !unzip nnsvs-english-support-v0.7.0.zip
  !mv nnsvs-english-support-*/ nnsvs-english-support-main/
  !cp /content/nnsvs-english-support-main/hed/DVX_English_NNSVS_MINIMAL.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-english-support-main/dic/blank.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_English_NNSVS_MINIMAL.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'ah', 'aa', 'ao', 'ae', 'ax', 'iy', 'ih', 'uh', 'uw', 'eh', 'er', 'ay', 'a', 'ey', 'e', 'oy', 'aw', 'ow', 'o', 'el', 'em', 'en'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_English_NNSVS_MINIMAL.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif Language == "DYVAUX Romance v0.1.0":
  !rm -rf nnsvs-romance-language-support-*
  !wget -O nnsvs-romance-language-support-v0.1.0.zip https://github.com/DYVAUX/nnsvs-romance-language-support/archive/refs/tags/v0.1.0.zip
  !unzip nnsvs-romance-language-support-v0.1.0.zip
  !mv nnsvs-romance-language-support-*/ nnsvs-romance-language-support-main/
  !cp /content/nnsvs-romance-language-support-main/hed/DVX_Romance.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-romance-language-support-main/dic/blank.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_Romance.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'e', 'i', 'o', 'u', 'aa', 'ao', 'au', 'oa', 'ax', 'eh', 'uu', 'uo', 'ui', 'eu', 'oe', 'er', 'ih', 'uh', 'an', 'en', 'in', 'on', 'un', 'rn'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_Romance.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/blank.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif Language == "DYVAUX Japanese v1.0.1":
  !rm -rf nnsvs-dyvaux-japanese-*
  !wget -O nnsvs-dyvaux-japanese-v1.0.1.zip https://github.com/DYVAUX/nnsvs-dyvaux-japanese/archive/refs/tags/v1.0.1.zip
  !unzip nnsvs-dyvaux-japanese-v1.0.1.zip
  !mv nnsvs-dyvaux-japanese-*/ nnsvs-dyvaux-japanese-main/
  !cp /content/nnsvs-dyvaux-japanese-main/hed/DVX_jp.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-dyvaux-japanese-main/dic/DVX_jp.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_jp.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/DVX_jp.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_jp.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/DVX_jp.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif Language == "DYVAUX Japanese MDN v1.0.1":
  !rm -rf nnsvs-dyvaux-japanese-*
  !wget -O nnsvs-dyvaux-japanese-v1.0.1.zip https://github.com/DYVAUX/nnsvs-dyvaux-japanese/archive/refs/tags/v1.0.1.zip
  !unzip nnsvs-dyvaux-japanese-v1.0.1.zip
  !mv nnsvs-dyvaux-japanese-*/ nnsvs-dyvaux-japanese-main/
  !cp /content/nnsvs-dyvaux-japanese-main/hed/DVX_jp_minimal.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-dyvaux-japanese-main/dic/DVX_jp.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_jp_minimal.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/DVX_jp.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/DVX_jp_minimal.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/DVX_jp.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "Default Japanese":
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/jp_qst_crazy_mono_013_enunu_221D.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/kana2phonemes_002_oto2lab.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/jp_qst_crazy_mono_013_enunu_221D.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/kana2phonemes_002_oto2lab.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "Default Japanese MDN":
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/jp_qst_crazy_mono_014_enunu_mdn_206D.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/kana2phonemes_002_oto2lab.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/jp_qst_crazy_mono_014_enunu_mdn_206D.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/kana2phonemes_002_oto2lab.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "SzTP's Polish v0.2.4":
  !rm -rf nnsvs-polish-support-*
  !wget -O nnsvs-polish-support-v0.2.4a.zip https://github.com/SzopaTatyJarka/nnsvs-polish-support/archive/refs/tags/0.2.4a.zip
  !unzip nnsvs-polish-support-v0.2.4a.zip
  !mv nnsvs-polish-support-*/ nnsvs-polish-support-main/
  !cp /content/nnsvs-polish-support-main/hed/sztj_polish.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-polish-support-main/dic/sztj_polish_phoneme.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/sztj_polish.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'A', 'a', 'E', 'e', 'i', 'o', 'u', 'y'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/sztj_polish_phoneme.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/sztj_polish.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/sztj_polish_phoneme.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "SzTP's Polish MDN v0.2.4":
  !rm -rf nnsvs-polish-support-*
  !wget -O nnsvs-polish-support-v0.2.4a.zip https://github.com/SzopaTatyJarka/nnsvs-polish-support/archive/refs/tags/0.2.4a.zip
  !unzip nnsvs-polish-support-v0.2.4a.zip
  !mv nnsvs-polish-support-*/ nnsvs-polish-support-main/
  !cp /content/nnsvs-polish-support-main/hed/sztj_polish_mdn.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-polish-support-main/dic/sztj_polish_phoneme.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/sztj_polish_mdn.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'A', 'a', 'E', 'e', 'i', 'o', 'u', 'y'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/sztj_polish_phoneme.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/sztj_polish_mdn.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/sztj_polish_phoneme.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "ArchiVoice Chinese v0.2.1":
  !rm -rf nnsvs-chinese-support-*
  !wget -O nnsvs-chinese-support-v0.2.1.zip https://github.com/Archivoice/nnsvs-chinese-support/archive/refs/tags/0.2.1.zip
  !unzip nnsvs-chinese-support-v0.2.1.zip
  !mv nnsvs-chinese-support-*/ nnsvs-chinese-support-main/
  !cp /content/nnsvs-chinese-support-main/hed/chinese.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-polish-support-main/dic/chinese.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/chinese.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'ia', 'ua', 'A', 'iA', 'uA', 'ng', 'N', 'e', 'ie', 'E', 'yu', 'i', 'I', 'ih', 'ii', 'U', 'uo', 'er', 'u', 'V', 'yE', 'uE', 'O', 'iO', 'yI', 'iU', 'uu'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/chinese.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/chinese.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/chinese.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "ArchiVoice Chinese MDN v0.2.1":
  !rm -rf nnsvs-chinese-support-*
  !wget -O nnsvs-chinese-support-v0.2.1.zip https://github.com/Archivoice/nnsvs-chinese-support/archive/refs/tags/0.2.1.zip
  !unzip nnsvs-chinese-support-v0.2.1.zip
  !mv nnsvs-chinese-support-*/ nnsvs-chinese-support-main/
  !cp /content/nnsvs-chinese-support-main/hed/chinese.hed /content/enunu_training_kit-main/train/hed/
  !cp /content/nnsvs-polish-support-main/dic/chinese.table /content/enunu_training_kit-main/train/dic/
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/chinese_MDN.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'ia', 'ua', 'A', 'iA', 'uA', 'ng', 'N', 'e', 'ie', 'E', 'yu', 'i', 'I', 'ih', 'ii', 'U', 'uo', 'er', 'u', 'V', 'yE', 'uE', 'O', 'iO', 'yI', 'iU', 'uu'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/chinese.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/chinese_MDN.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/chinese.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml
elif  Language == "Custom":
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/YOUR_CUSTOM_HED.hed|g' /content/enunu_training_kit-main/train/enuconfig.yaml
  !sed -i -r "s|(VOWELS = )(.+)|\1{'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'}|g" /content/enunu_training_kit-main/train/stage0/compare_mono_align_and_mono_score.py
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/YOUR_CUSTOM_DIC.table|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/YOUR_CUSTOM_HED.hed|g' /content/enunu_training_kit-main/train/config.yaml
  !sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/YOUR_CUSTOM_DIC.table|g' /content/enunu_training_kit-main/train/enuconfig.yaml

if Vibrato_Mode == "none":
  !sed -i -r 's|(stream_sizes:)(\s+)(.+)|\1\2\[180, 3, 1, 15\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(has_dynamic_features:)(\s+)(.+)|\1\2\[true, true, false, true\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(out_dim:)(\s+)(.+)|\1\2199|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(vibrato_mode:)(\s+)(.+)|\1\2none|g' /usr/local/lib/python3.7/dist-packages/nnsvs/bin/conf/prepare_features/acoustic/static_deltadelta.yaml
elif Vibrato_Mode == "diff":
  !sed -i -r 's|(stream_sizes:)(\s+)(.+)|\1\2\[180, 3, 1, 15, 3\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(has_dynamic_features:)(\s+)(.+)|\1\2\[true, true, false, true, true\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(out_dim:)(\s+)(.+)|\1\2202|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(vibrato_mode:)(\s+)(.+)|\1\2diff|g' /usr/local/lib/python3.7/dist-packages/nnsvs/bin/conf/prepare_features/acoustic/static_deltadelta.yaml
elif Vibrato_Mode == "sine":
  !sed -i -r 's|(stream_sizes:)(\s+)(.+)|\1\2\[180, 3, 1, 15, 6, 1\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(has_dynamic_features:)(\s+)(.+)|\1\2\[true, true, false, true, true, false\]|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(out_dim:)(\s+)(.+)|\1\2206|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
  !sed -i -r 's|(vibrato_mode:)(\s+)(.+)|\1\2sine|g' /usr/local/lib/python3.7/dist-packages/nnsvs/bin/conf/prepare_features/acoustic/static_deltadelta.yaml

print('Done!')

In [ ]:
#@title Standard Training Parameters:
#@markdown RUN [**THIS CELL**](#scrollTo=xMjpRZSk7dTA) OR THIS CELL WILL NOT WORK.
#@markdown <br> "resume_from_checkpoint" enables resuming training or starting from a pretrained model.

resume_folder = "" #@param {type:"string"}
#@markdown Set `resume_folder` blank to remove the pretrained model.
segment_mode = "middle" #param ["middle", "short"]
middle_frequency =  2#@param {type:"integer"}
#@markdown Setting a smaller `middle_frequency` may improve training if time between pau is long.
low_storage = True #@param {type:"boolean"}
#@markdown `low_storage` disables checkpoints to save storage space for free users,
#@markdown ### <br> Acoustic Settings:
acoustic_epochs = 60 #@param {type:"integer"}
acoustic_hidden_dim = 256 #@param {type:"slider", min:0, max:1024, step:128}
#@markdown Set Conv1dResnetMDN to NO MORE than 384 hidden_dim.
acoustic_num_layers = 4 #@param {type:"slider", min:2, max:6, step:1}
acoustic_dropout = 0.1 #@param {type:"slider", min:0, max:0.8, step:0.05}
acoustic_batch_size =  8 #@param {type:"integer"}
acoustic_feats_criterion = "mse" #@param ["mse", "mae"]
# acoustic_eval_checkpoint = "best_loss.pth (default)" #@param ["best_loss.pth (default)", "latest.pth"]

#@markdown ### <br> Duration Settings:
duration_epochs = 200 #@param {type:"integer"}
duration_hidden_dim = 256 #@param ["128", "256"] {type:"raw"}
duration_num_layers = 3 #@param ["3", "4"] {type:"raw"}
duration_dropout = 0.5 #@param {type:"slider", min:0, max:0.8, step:0.05}
duration_batch_size = 8 #@param {type:"integer"}
duration_feats_criterion = "mse" #@param ["mse", "mae"]
# duration_eval_checkpoint = "best_loss.pth (default)" #@param ["best_loss.pth (default)", "latest.pth"]

#@markdown ### <br> Timelag Settings:
timelag_epochs = 200 #@param {type:"integer"}
timelag_hidden_dim = 128 #@param ["128", "256"] {type:"raw"}
timelag_num_layers = 3 #@param ["3", "4"] {type:"raw"}
timelag_dropout = 0.5 #@param {type:"slider", min:0, max:0.8, step:0.05}
timelag_batch_size = 8 #@param {type:"integer"}
timelag_feats_criterion = "mse" #@param ["mse", "mae"]

# lr_scheduler = "StepLR (default)" #@param ["StepLR (default)", "ReduceLROnPlateau"]
# #@markdown Don't change lr_scheduler if you don't know what you're doing.

import math
# checkpoint resume stuff
!sed -i -r '/#/!s|(pretrained_expdir:)(.*)|\1 "{resume_folder}"|g' /content/enunu_training_kit-main/train/config.yaml
# Segmentation stuff
!sed -i -r 's|(segmentation_mode:)(\s+)(.+)|\1\2{segment_mode}|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(middle_frequency:)(\s+)(.+)|\1\2{middle_frequency}|g' /content/enunu_training_kit-main/train/config.yaml
# Acoustic config
!sed -i -r 's|(batch_size:)(\s+)(.+)|\1\2{acoustic_batch_size}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/data/myconfig.yaml
!sed -i -r 's|(  hidden_dim:)(\s+)(.+)|\1\2{acoustic_hidden_dim}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
!sed -i -r 's|(num_layers:)(\s+)(.+)|\1\2{acoustic_num_layers}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
!sed -i -r 's|(dropout:)(\s+)(.+)|\1\2{acoustic_dropout}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/model/acoustic_custom.yaml
!sed -i -r 's|(nepochs:)(\s+)(.+)|\1\2{acoustic_epochs}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
!sed -i -r 's|(feats_criterion:)(\s+)(.+)|\1\2{acoustic_feats_criterion}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml
acoustic_checkpoint_interval = acoustic_epochs * 0.15
acoustic_checkpoint_interval_trunc = math.trunc(acoustic_checkpoint_interval)
acoustic_checkpoint_interval_trunc_min = min(acoustic_checkpoint_interval_trunc, 80)
if low_storage:
  acoustic_checkpoint_interval_trunc_min = 999999
!sed -i -r 's|(checkpoint_epoch_interval:)(\s+)(.+)|\1\2{acoustic_checkpoint_interval_trunc_min}|g' /content/enunu_training_kit-main/train/conf/train/acoustic/train/myconfig.yaml

#Duration config
!sed -i -r 's|(batch_size:)(\s+)(.+)|\1\2{duration_batch_size}|g' /content/enunu_training_kit-main/train/conf/train/duration/data/myconfig.yaml
!sed -i -r 's|(  hidden_dim:)(\s+)(.+)|\1\2{duration_hidden_dim}|g' /content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml
!sed -i -r 's|(  num_layers:)(\s+)(.+)|\1\2{duration_num_layers}|g' /content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml
!sed -i -r 's|(  dropout:)(\s+)(.+)|\1\2{duration_dropout}|g' /content/enunu_training_kit-main/train/conf/train/duration/model/duration_custom.yaml
!sed -i -r 's|(nepochs:)(\s+)(.+)|\1\2{duration_epochs}|g' /content/enunu_training_kit-main/train/conf/train/duration/train/myconfig.yaml
!sed -i -r 's|(checkpoint_epoch_interval:)(\s+)(.+)|\1\2{duration_checkpoint_interval}|g' /content/enunu_training_kit-main/train/conf/train/duration/train/myconfig.yaml
!sed -i -r 's|(feats_criterion:)(\s+)(.+)|\1\2{duration_feats_criterion}|g' /content/enunu_training_kit-main/train/conf/train/duration/train/myconfig.yaml
duration_checkpoint_interval = duration_epochs * 0.15
duration_checkpoint_interval_trunc = math.trunc(duration_checkpoint_interval)
duration_checkpoint_interval_trunc_min = min(duration_checkpoint_interval_trunc, 80)
if low_storage:
  acoustic_checkpoint_interval = 0
!sed -i -r 's|(checkpoint_epoch_interval:)(\s+)(.+)|\1\2{duration_checkpoint_interval_trunc_min}|g' /content/enunu_training_kit-main/train/conf/train/duration/train/myconfig.yaml

#Timelag config
!sed -i -r 's|(batch_size:)(\s+)(.+)|\1\2{timelag_batch_size}|g' /content/enunu_training_kit-main/train/conf/train/timelag/data/myconfig.yaml
!sed -i -r 's|(  hidden_dim:)(\s+)(.+)|\1\2{timelag_hidden_dim}|g' /content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml
!sed -i -r 's|(  num_layers:)(\s+)(.+)|\1\2{timelag_num_layers}|g' /content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml
!sed -i -r 's|(  dropout:)(\s+)(.+)|\1\2{timelag_dropout}|g' /content/enunu_training_kit-main/train/conf/train/timelag/model/timelag_custom.yaml
!sed -i -r 's|(nepochs:)(\s+)(.+)|\1\2{timelag_epochs}|g' /content/enunu_training_kit-main/train/conf/train/timelag/train/myconfig.yaml
!sed -i -r 's|(checkpoint_epoch_interval:)(\s+)(.+)|\1\2{timelag_checkpoint_interval}|g' /content/enunu_training_kit-main/train/conf/train/timelag/train/myconfig.yaml
!sed -i -r 's|(feats_criterion:)(\s+)(.+)|\1\2{timelag_feats_criterion}|g' /content/enunu_training_kit-main/train/conf/train/timelag/train/myconfig.yaml
timelag_checkpoint_interval = timelag_epochs * 0.15
timelag_checkpoint_interval_trunc = math.trunc(timelag_checkpoint_interval)
timelag_checkpoint_interval_trunc_min = min(timelag_checkpoint_interval_trunc, 80)
if low_storage:
  acoustic_checkpoint_interval = 0
!sed -i -r 's|(checkpoint_epoch_interval:)(\s+)(.+)|\1\2{timelag_checkpoint_interval_trunc_min}|g' /content/enunu_training_kit-main/train/conf/train/timelag/train/myconfig.yaml


In [ ]:
#@title (optional) Train from a pretrained model. (Not implemented)

#@markdown You can improve your result by training your dataset on top of a pretrained model.
#@markdown <br/>At this time there are only models available for Japanese
#@markdown <br/>Note that pretraining for MDN duration and timelag models may break on small dataset.
#@markdown <br/>**This cell will override all previous settings to ensure training works smoothly.**
Voice = "JP_Ritsu" #@param ["JP_Ritsu", "EN_DYVAUX_Colton"]
Acoustic_Model = "Conv1dResnet" #@param ["Conv1dResnet", "Conv1dResnetMDN(rMDN)", "MDN", "RMDN", "ResF0Conv1dResnet", "ResF0Conv1dResnetMDN", "ResSkipF0FFConvLSTM", "LSTMRNN", "FeedForwardNet"]
Duration_Model = "LSTMRNN" #@param ["LSTMRNN", "MDN"]
Timelag_Model = "MDN" #@param ["FeedForwardNet", "MDN"]

skip_duration_training = False #@param {type:"boolean"}
#@markdown `skip_duration_training` allows you to skip the duration and timelag training steps to hopefully avoid those models from breaking with small datasets

# Training

---



In [ ]:
#@title Tensorboard
#@markdown Allows you to view training info. enable refreshing in the settings ("gear" icon)

import datetime, os
%tensorboard --load_fast=true --reload_interval=1 --reload_multifile=true --logdir=/content/enunu_training_kit-main/train/tensorboard/

In [ ]:
#@title Step 3: Begin training
starting_stage = 0 #@param {type:"slider", min:0, max:7, step:1.0}
stopping_stage = 7 #@param {type:"slider", min:0, max:7, step:1.0}
singer_name = "Unnamed" #@param {type:"string"}
#@markdown `singer_name:` should use underscores and not spaces (_)
!sed -i -r 's|(spk:)(\s+)(.+)|\1\2{singer_name}|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(spk:)(\s+)(.+)|\1\2{singer_name}|g' /content/enunu_training_kit-main/train/config_rmdn.yaml
!sed -i -r 's|(model_dir:)(\s+)(.+)|\1\2exp/{singer_name}\_dyvaux_notebook|g' /content/enunu_training_kit-main/train/enuconfig.yaml
!sed -i -r 's|(tag:)(\s+)(.+)|\1\2dyvaux_notebook|g' /content/enunu_training_kit-main/train/config.yaml
!sed -i -r 's|(stats_dir:)(\s+)(.+)|\1\2dump/{singer_name}/norm|g' /content/enunu_training_kit-main/train/enuconfig.yaml

%cd "/content/enunu_training_kit-main/train"
#!rm -rf data #remove this line if you want to continue training along with modifying the stage
!bash run.sh --stage $starting_stage --stop_stage $stopping_stage

In [ ]:
#@title Step 4: Package and (optionally) store your model onto your Google Drive

Store_on_Google_Drive = False #@param {type:"boolean"}

Clean_up_model = True #@param {type:"boolean"}
#@markdown `Clean_up_model` deletes extra checkpoints to reduce singer size. Does not affect quality.

singer_name = "Unnamed" #@param {type:"string"}
#@markdown `singer_name` should match the name used during Step 3.

import subprocess
time = subprocess.getoutput('date +%y%m%d-%H%M-UTC')

if Clean_up_model == True:
  %rm -f /content/enunu_training_kit-main/train/release/{singer_name}_---/exp/{singer_name}_dyvaux_notebook/*/checkpoint*
  %rm -f /content/enunu_training_kit-main/train/release/{singer_name}_---/exp/{singer_name}_dyvaux_notebook/*/epoch*
#  %rm -f /content/enunu_training_kit-main/train/release/{singer_name}_---/exp/{singer_name}_dyvaux_notebook/*/latest*

#!touch "/content/enunu_training_kit-main/train/release/{singer_name}_---/oto.ini"
%cd -q "/content/enunu_training_kit-main/train/release/"
print('\033[97;100m' + 'Compressing Model' + '\033[0m')
!7za -bso0 a "/content/{singer_name}_NNSVS_model_{time}.7z" "{singer_name}_---"
if Store_on_Google_Drive == False:
  print('Done!')

if Store_on_Google_Drive == True:
  %mkdir -p /content/drive/MyDrive/NNSVS_Release_Models
  print('\033[97;100m' + 'Copying Singer to your Google Drive' + '\033[0m')
  !pv /content/{singer_name}_NNSVS_model_{time}.7z > /content/drive/MyDrive/NNSVS_Release_Models/{singer_name}_NNSVS_model_{time}.7z
  print('Done!')